Este exemplo é baseado nos tutoriais disponíveis na página do TensorFlow (https://www.tensorflow.org/get_started/mnist/beginners e https://www.tensorflow.org/get_started/mnist/pros)
e também no TensorBoard https://www.youtube.com/watch?v=eBbEDRsCmv4 

In [25]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
#import TensorFlow and MNIST
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

# carrega os dados do MNIST com os labels no formato "one-hot vector"
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [26]:
# definimos algumas constantes
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
IMAGE_CHANNELS = 1
IMAGE_SIZE = IMAGE_WIDTH * IMAGE_HEIGHT * IMAGE_CHANNELS

N_CLASSES = 10
MAX_ITERS = 201
BATCH_SIZE = 64
LOGDIR='/home/vitor/mnist/'
if not os.path.exists(LOGDIR):
    os.makedirs(LOGDIR)
n_samples = 1024

In [27]:
from urllib.request import urlretrieve

GITHUB_URL ='https://raw.githubusercontent.com/ejulio/talks/master/desmistificando-o-deep-learning-com-tensorflow/'

### baixa sprites e labels para o projetor de embeddings ###
urlretrieve(GITHUB_URL + 'labels_1024.tsv', LOGDIR + 'labels_1024.tsv')
urlretrieve(GITHUB_URL + 'sprite_1024.png', LOGDIR + 'sprite_1024.png')

('/home/vitor/mnist/sprite_1024.png',
 <http.client.HTTPMessage at 0x7fdb4e32deb8>)

In [28]:
tf.reset_default_graph()
sess = tf.Session()
with tf.name_scope('input'):
    # um placeholder para entradas (imagens) shape=(BATCH_SIZE, n_features)
    x = tf.placeholder(tf.float32, shape=[None, IMAGE_SIZE], name='images')
    # um placeholder para os labels, shape = (BATCH_SIZE, n_classes)
    y = tf.placeholder(tf.float32, shape=[None, N_CLASSES], name="labels")

    # pesos W
    W = tf.Variable(
        tf.truncated_normal([IMAGE_SIZE, N_CLASSES], stddev=0.05),
        name='Weights')
    # viéses b
    b = tf.Variable(tf.zeros([N_CLASSES]), name='Biases')

    tf.summary.histogram("weights", W)
    tf.summary.histogram("biases", b)

    # modelo linear
    logits = tf.matmul(x, W) + b
    
with tf.name_scope('loss'):
    loss = loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    tf.summary.scalar("loss", loss)

with tf.name_scope("train"):
    # SGD + momentum para otimização
    learning_rate = 0.01
    momentum = 0.9
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    # minimiza o valor da loss
    optimizer = optimizer.minimize(loss)
    
with tf.name_scope("test_accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    test_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("test_accuracy", test_accuracy)

summ = tf.summary.merge_all()    

with tf.name_scope("embeddings"):
    embedding_input = x
    embedding_size = IMAGE_SIZE
    embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + 'linear_regression_emb')
    writer.add_graph(sess.graph)

    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = LOGDIR + 'sprite_1024.png'
    embedding_config.metadata_path = LOGDIR + 'labels_1024.tsv'
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)
    
for iteration in range(MAX_ITERS):
    # obtém o próximo batch de imagens e labels
    (images, labels) = mnist.train.next_batch(BATCH_SIZE)
    
    # executa uma iteração da otimização
    (_, iter_loss) = sess.run([optimizer, loss], 
                          feed_dict={x: images,
                                     y: labels}) 
    
    # avalia o modelo a cada 10 iterações
    if iteration % 10 == 0:
        [test_acc, s] = sess.run([test_accuracy, summ], 
                                 feed_dict={x: mnist.test.images[:n_samples], 
                                            y: mnist.test.labels[:n_samples]})
        print('Iteration {}, loss = {}.'.format(iteration, iter_loss))
        print('Test Accuracy: {0}'.format(test_acc))
        writer.add_summary(s, iteration)
                                     
    if iteration % 100 == 0:
        sess.run(assignment, feed_dict={x: mnist.test.images[:n_samples], 
                                        y: mnist.test.labels[:n_samples]})
        saver.save(sess, os.path.join(LOGDIR, "linear_regression.ckpt"), iteration)                                        
                                     
                                    

Iteration 0, loss = 2.3927762508392334.
Test Accuracy: 0.1220703125
Iteration 10, loss = 1.9464082717895508.
Test Accuracy: 0.4208984375
Iteration 20, loss = 1.4002333879470825.
Test Accuracy: 0.70703125
Iteration 30, loss = 1.144566297531128.
Test Accuracy: 0.7333984375
Iteration 40, loss = 0.8493523597717285.
Test Accuracy: 0.771484375
Iteration 50, loss = 0.8042867183685303.
Test Accuracy: 0.80078125
Iteration 60, loss = 0.7084593176841736.
Test Accuracy: 0.8076171875
Iteration 70, loss = 0.5597255229949951.
Test Accuracy: 0.8193359375
Iteration 80, loss = 0.5109997987747192.
Test Accuracy: 0.8291015625
Iteration 90, loss = 0.7448969483375549.
Test Accuracy: 0.8408203125
Iteration 100, loss = 0.4932386577129364.
Test Accuracy: 0.845703125
Iteration 110, loss = 0.5250451564788818.
Test Accuracy: 0.853515625
Iteration 120, loss = 0.6671329736709595.
Test Accuracy: 0.8486328125
Iteration 130, loss = 0.784690260887146.
Test Accuracy: 0.8505859375
Iteration 140, loss = 0.7149437665939331

In [30]:
# Adiciona Convolução
def conv_layer(input, size_in, size_out, name="conv", name2="pool"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.05), name="Weights")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="Biases")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        relu = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", relu)

        return relu

# Adiciona Max Pooling
def max_pool(input, name):
    with tf.name_scope(name):
        return tf.nn.max_pool(input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# Adiciona totalmente conectada
def fc_layer(input, size_in, size_out, name="fc"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.05), name="Weights")
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="Biases")
        relu = tf.nn.relu(tf.matmul(input, w) + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", relu)

        return relu


tf.reset_default_graph()
sess = tf.Session()

# Setup placeholders
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 784], name="images")
    y = tf.placeholder(tf.float32, shape=[None, 10], name="labels")

# Redimensiona a imagem
with tf.name_scope('input_reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', x_image, 10)

# convoluções e max poolings
conv1 = conv_layer(x_image, 1, 32, "conv1", "pool1")
conv1 = max_pool(conv1, "pool1")
conv2 = conv_layer(conv1, 32, 64, "conv2", "pool2")
conv2 = max_pool(conv2, "pool2")

# transforma a imagem em um array
with tf.name_scope('flatten'):
    flattened = tf.reshape(conv2, [-1, 7 * 7 * 64])

# totalmente conectada 1
fc1 = fc_layer(flattened, 7 * 7 * 64, 1024, "fc1")

with tf.name_scope('dropout'):
    # dropout: elimina algumas unidades; veremos no final se der tempo
    keep_prob = tf.placeholder(tf.float32)
    fc1 = tf.nn.dropout(fc1, keep_prob)

# totalmente conectada 2
fc2 = fc_layer(fc1, 1024, 10, "fc2")

with tf.name_scope("loss"):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=fc2, labels=y), name="cross_entropy")
    tf.summary.scalar("loss", loss)

with tf.name_scope("train"):
    # SGD + momentum para otimização
    learning_rate = 0.01
    momentum = 0.9
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
    # minimiza o valor da loss
    optimizer = optimizer.minimize(loss)

with tf.name_scope("test_accuracy"):
    correct_prediction = tf.equal(tf.argmax(fc2, 1), tf.argmax(y, 1))
    test_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("test_accuracy", test_accuracy)

summ = tf.summary.merge_all()    
    
with tf.name_scope("embeddings"):
    embedding_input = fc1
    embedding_size = 1024
    embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + 'cnn_emb')
    writer.add_graph(sess.graph)

    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    embedding_config.sprite.image_path = LOGDIR + 'sprite_1024.png'
    embedding_config.metadata_path = LOGDIR + 'labels_1024.tsv'
    embedding_config.sprite.single_image_dim.extend([28, 28])
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)

for iteration in range(MAX_ITERS):
    # obtém o próximo batch de imagens e labels
    (images, labels) = mnist.train.next_batch(BATCH_SIZE)
    
    # executa uma iteração da otimização
    (_, iter_loss) = sess.run([optimizer, loss], 
                          feed_dict={x: images,
                                     y: labels,
                                     keep_prob: 0.5})  
    
    # avalia o modelo a cada 10 iterações
    if iteration % 10 == 0:
        [test_acc, s] = sess.run([test_accuracy, summ], 
                                 feed_dict={x: mnist.test.images[:n_samples], 
                                            y: mnist.test.labels[:n_samples],
                                            keep_prob: 1.0})
        print('Iteration {}, loss = {}.'.format(iteration, iter_loss))
        print('Test Accuracy: {0}'.format(test_acc))
        writer.add_summary(s, iteration)
        
    if iteration % 100 == 0:
        sess.run(assignment, feed_dict={x: mnist.test.images[:n_samples], 
                                        y: mnist.test.labels[:n_samples],
                                        keep_prob: 1.0})
        saver.save(sess, os.path.join(LOGDIR, "cnn.ckpt"), iteration)  

Iteration 0, loss = 2.388707399368286.
Test Accuracy: 0.0693359375
Iteration 10, loss = 2.28298282623291.
Test Accuracy: 0.099609375
Iteration 20, loss = 2.23799467086792.
Test Accuracy: 0.2060546875
Iteration 30, loss = 2.201395034790039.
Test Accuracy: 0.2705078125
Iteration 40, loss = 2.1243033409118652.
Test Accuracy: 0.3828125
Iteration 50, loss = 1.5882071256637573.
Test Accuracy: 0.453125
Iteration 60, loss = 1.8074626922607422.
Test Accuracy: 0.494140625
Iteration 70, loss = 1.508946418762207.
Test Accuracy: 0.525390625
Iteration 80, loss = 1.5316241979599.
Test Accuracy: 0.62890625
Iteration 90, loss = 0.9368271827697754.
Test Accuracy: 0.7412109375
Iteration 100, loss = 0.6351594924926758.
Test Accuracy: 0.748046875
Iteration 110, loss = 0.8418694734573364.
Test Accuracy: 0.7470703125
Iteration 120, loss = 0.8123348355293274.
Test Accuracy: 0.779296875
Iteration 130, loss = 0.7865946292877197.
Test Accuracy: 0.8310546875
Iteration 140, loss = 0.6085005402565002.
Test Accuracy